In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import torch
import torch.nn as nn
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd

2025-05-08 16:21:48.854891: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 16:21:49.823784: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 16:21:50.606310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746735711.200515    5887 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746735711.367272    5887 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746735712.979851    5887 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
image_dir = '/scratch/ac9743/pytorch-example/tier1/images'
label_dir = '/scratch/ac9743/pytorch-example/tier1/labels'

df = pd.read_csv("/scratch/ac9743/pytorch-example/tier1.csv")

# For the GAN we only care about post disaster images
filtered_df = df[
    (df["stage"] == "post") &
    (df["feature_type"] == "building")
]

num_disaster_types = len(filtered_df["disaster_type"].unique())

print(len(filtered_df))

97851


In [3]:
import os
import json
import numpy as np
import tifffile
from PIL import Image, ImageDraw
from shapely import wkt
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, metadata_df, image_dir, label_dir, image_size=(256, 256)): 
        self.metadata_df = metadata_df
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_size = image_size

    def __len__(self):
        return len(self.metadata_df)

    def preprocess_image(self, image):
        # Normalize image
        image = image.resize(self.image_size)
        image = np.array(image).astype(np.float32) / 255.0  # Normalize to [0, 1]
        return torch.from_numpy(image).permute(2, 0, 1)  # Convert to tensor and rearrange dims to [C, H, W]

    def preprocess_mask(self, mask):
        mask = mask.resize(self.image_size)
        mask = np.array(mask).astype(np.float32)
        return torch.from_numpy(mask).unsqueeze(0)  # Shape [1, H, W]

    def __getitem__(self, idx):
        row = self.metadata_df.iloc[idx]
        img_path = os.path.join(self.image_dir, row["image_filename"])
        label_path = os.path.join(self.label_dir, row["label_filename"])

        # Load image using tifffile
        image = tifffile.imread(img_path)
        if image.dtype == np.int16:
            image = np.clip(image, 0, 255).astype(np.uint8)
        image = Image.fromarray(image).convert("RGB")

        # Load mask from polygons
        mask = Image.new("L", self.image_size, 0)
        draw = ImageDraw.Draw(mask)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        features = label_data.get("features", {}).get("lng_lat", [])
        for feature in features:
            wkt_str = feature.get("wkt")
            try:
                poly = wkt.loads(wkt_str)
                if poly.is_valid:
                    coords = [(x, y) for x, y in poly.exterior.coords]
                    draw.polygon(coords, outline=1, fill=1)
            except Exception as e:
                print(f"Polygon error in {label_path}: {e}")

        # Preprocess image and mask
        image_tensor = self.preprocess_image(image)
        mask_tensor = self.preprocess_mask(mask)

        return {
            "image": image_tensor,        
            "mask": mask_tensor,          
            "stage": row["stage"],
            "disaster_type": row["disaster_type"]
        }

In [4]:
dataset = CustomDataset(filtered_df, image_dir, label_dir)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
discriminator = keras.Sequential([
    keras.Input(shape=(256, 256, 3)),
    
    layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),  # 128x128
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),  # 64x64
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),  # 32x32
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2D(512, kernel_size=4, strides=2, padding="same"),  # 16x16
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2D(512, kernel_size=4, strides=2, padding="same"),  # 8x8
    layers.LeakyReLU(alpha=0.2),
    
    layers.Flatten(),
    layers.Dense(1, activation="sigmoid")  # Or linear if using WGAN
], name="discriminator")


/home/ac9743/.local/lib/python3.9/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
E0000 00:00:1746735769.229576    5887 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1746735769.233423    5887 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
latent_dim = 128

generator = keras.Sequential([
    keras.Input(shape=(latent_dim,)),
    layers.Dense(8 * 8 * 256),
    layers.Reshape((8, 8, 256)),
    
    # Increase the number of upsampling layers to output 256x256
    layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),  # 16x16
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),  # 32x32
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),   # 64x64
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding="same"),   # 128x128
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2DTranspose(16, kernel_size=4, strides=2, padding="same"),   # 256x256
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2D(3, kernel_size=3, activation="tanh", padding="same")  # Output 256x256x3
], name="generator")


In [7]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim))
        generated_images = self.generator(random_latent_vectors)
        combined_images = tf.concat([generated_images, real_images], axis=0)
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))],
            axis=0
        )

        # Add noise to discriminator labels to make it's job harder
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim))

        misleading_labels = tf.zeros((batch_size, 1))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(
                self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(
            zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(),
                "g_loss": self.g_loss_metric.result()}

In [8]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
    
        generated_images = (generated_images + 1) * 127.5

        # Create a directory to store generated images if it doesn't exist
        os.makedirs("generated_images_256", exist_ok=True)

        for i in range(self.num_img):
            img = keras.utils.array_to_img(generated_images[i])
            img.save(f"generated_images_256/generated_img_{epoch:03d}_{i}.png")

In [9]:
gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

In [10]:
epochs = 25
batch_size = 32

# Converting to a tensor dataframe because GAN model is in tensor/keras

def preprocess_image_tf(item):
    return tf.convert_to_tensor(item["image"], dtype=tf.float32)

tf_dataset = tf.data.Dataset.from_generator(
    lambda: ({"image": tf.transpose(d["image"], (1, 2, 0))} for d in dataset),
    output_signature={
        "image": tf.TensorSpec(shape=(256, 256, 3), dtype=tf.float32),
    }
)

tf_dataset = tf_dataset.map(lambda x: x["image"]).batch(batch_size)
tf_dataset = tf_dataset.repeat()

In [11]:
_ = generator(tf.random.normal((1, latent_dim)))
_ = discriminator(tf.random.normal((1, 256, 256, 3)))

In [ ]:
checkpoint = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator,
    gan=gan,
    g_optimizer=gan.g_optimizer,  # Use the optimizers from your GAN model
    d_optimizer=gan.d_optimizer
)

ckpt_manager = tf.train.CheckpointManager(checkpoint, './checkpoints', max_to_keep=3)

if ckpt_manager.latest_checkpoint:
    checkpoint.restore(ckpt_manager.latest_checkpoint)
    print("Restored from", ckpt_manager.latest_checkpoint)
else:
    print("Initializing from scratch.")

class CheckpointSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
    
steps_per_epoch = len(dataset) // batch_size 

gan.fit(
    tf_dataset,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim), CheckpointSaver()]
)

Restored from ./checkpoints/ckpt-39
Epoch 1/25
  64/3057 ━━━━━━━━━━━━━━━━━━━━ 5:16:56 6s/step - d_loss: 0.1914 - g_loss: 3.5163